In [1]:
import numpy as np

In [2]:
class Unit:
    def __init__(self, value, grad):
        self.value = value
        self.grad = grad

In [3]:
class AddGate:
    def __init__(self):
        self.x = None
        self.y = None
        self.z = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        self.z = Unit(self.x.value + self.y.value, 0.0)
        return self.z
    def backward(self):
        self.x.grad += 1 * self.z.grad
        self.y.grad += 1 * self.z.grad

In [4]:
class MulGate:
    def __init__(self):
        self.x = None
        self.y = None
        self.z = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        self.z = Unit(self.x.value * self.y.value, 0.0)
        return self.z
    def backward(self):
        self.x.grad += self.y.value * self.z.grad
        self.y.grad += self.x.value * self.z.grad

In [5]:
class SigmoidGate:
    def __init__(self):
        self.x = None
        self.z = None
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    def forward(self, x):
        self.x = x
        self.z = Unit(SigmoidGate.sigmoid(self.x.value), 0.0)
        return self.z
    def backward(self):
        s = self.z.value
        self.x.grad += s * (1-s) * self.z.grad

In [7]:
# value and gates definition
w0 = Unit(2.0, 0.0)
w1 = Unit(-3.0, 0.0)
w2 = Unit(-3.0, 0.0)
x = Unit(-1.0, 0.0)
y = Unit(-2.0, 0.0)

mulg0 = MulGate()
mulg1 = MulGate()
addg0 = AddGate()
addg1 = AddGate()
sg0 = SigmoidGate()

In [8]:
# forward pass
def forwardNetwork():
    w0x = mulg0.forward(w0, x)
    w1y = mulg1.forward(w1, y)
    w0xpw1y = addg0.forward(w0x, w1y)
    w0xpw1ypw2 = addg1.forward(w0xpw1y, w2)
    return  sg0.forward(w0xpw1ypw2)

s = forwardNetwork()
print('network output: ' + str(s.value))

network output: 0.73105857863


In [9]:
# backward pass
def backwardNetwork():
    s.grad = 1.0;
    sg0.backward()
    addg1.backward()
    addg0.backward()
    mulg1.backward()
    mulg0.backward()

backwardNetwork()

In [14]:
# one step of gradient descent
step_size = 0.01;
w0.value += step_size * w0.grad 
w1.value += step_size * w1.grad
w2.value += step_size * w2.grad
x.value += step_size * x.grad
y.value += step_size * y.grad

s = forwardNetwork()
print('network output after one step of gradient descent: ' + str(s.value))

network output after one step of gradient descent: 0.766234162642
